In [1]:
#Basic imports 
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

#NLP imports 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import spacy
spacy.load('en_core_web_sm')

# Feature Creations 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Dimension Reduction 
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import Normalizer

#Model Imports 
from sklearn import ensemble
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#Time
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shuaix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('https://www.dropbox.com/s/d4ye48a67tth2ae/Reviews.csv?dl=1')

In [3]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
#drop uneccessary columns 
df.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Time', 'Summary'],axis=1,inplace=True)
df.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 2 columns):
Score    568454 non-null int64
Text     568454 non-null object
dtypes: int64(1), object(1)
memory usage: 8.7+ MB


In [6]:
df.isnull().sum()

Score    0
Text     0
dtype: int64

In [7]:
# Remove duplicate reviews  
df.drop_duplicates(subset=['Score','Text'],keep='first',inplace=True)

In [8]:
df.Score.value_counts()

5    250745
4     56074
1     36280
3     29772
2     20804
Name: Score, dtype: int64

In [9]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    
    text = re.sub(r'--',' ',text)
    
    # Removes hyperlinks 
    text = re.sub(r'<a\s+href=(?:"([^"]+)"|\'([^\']+)\').*?>(.*?)</a>',' ', text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    text = text.lower()
    
    return text

In [10]:
df['Clean'] = df['Text'].apply(text_cleaner)

In [11]:
df.head()

,Score,Text,Clean
0,5,I have bought several of the Vitality canned d...,i have bought several of the vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled as jumbo salted peanut...
2,4,This is a confection that has been around a fe...,this is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...,if you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...,great taffy at a great price. there was a wide...


In [12]:
# Declare X and Y values
X = df.Clean
y = df.Score

In [13]:
#split out training/test data 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

#split training set again to test best params with reduce execution times 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train,y_train, test_size=0.8, random_state=0)

In [14]:
# Display the shape to make sure the length is the same.
print(X_train2.shape)
print(y_train2.shape)

(62988,)
(62988,)


In [15]:
# Lemmatize our text reviews to limit variations on same words

stop_words = set(stopwords.words('english'))
lemmatizer = spacy.lang.en.English()

def lemmatize_sen(text):
    tokens = lemmatizer(text) 
    return([token.lemma_ for token in tokens if token not in stop_words])

In [16]:
# Setting max features to 1000 will choose the 1000 with the highest count.
vectorizer = CountVectorizer(max_features=1000, tokenizer=lemmatize_sen)
# Train the model and transform it
X_train2_matrix = vectorizer.fit_transform(X_train2)

X_train2_matrix

<62988x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 2994857 stored elements in Compressed Sparse Row format>

In [17]:
# X_test
X_test_matrix = vectorizer.transform(X_test)
X_test_matrix

<78735x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 3727550 stored elements in Compressed Sparse Row format>

## Logistic regression

In [18]:
start_time = time.time()
param_dist = {'penalty':['l1','l2'],
                'C':[1,100,1000]}

lr = LogisticRegression()

random_search = RandomizedSearchCV(LogisticRegression(class_weight = 'balanced',random_state=0, n_jobs= -1), param_distributions=param_dist, n_iter=5)
#Fit the Data
random_search.fit(X_train2_matrix, y_train2)
print(random_search.score(X_test_matrix, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to

0.6808534959039817
-- Execution time: 349.23294496536255 seconds ---


In [19]:
random_search.best_params_

{'penalty': 'l1', 'C': 1}

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, random_search.predict(X_test_matrix)))

              precision    recall  f1-score   support

           1       0.52      0.63      0.57      7335
           2       0.22      0.25      0.23      4160
           3       0.31      0.33      0.32      5980
           4       0.39      0.27      0.32     11089
           5       0.84      0.86      0.85     50171

   micro avg       0.68      0.68      0.68     78735
   macro avg       0.46      0.47      0.46     78735
weighted avg       0.67      0.68      0.67     78735



## Random forest

In [21]:
start_time = time.time()
param_dist = {'n_estimators':[500,700,800],
                'criterion':['entropy'],
                'max_depth':[6,8,10],
                'min_samples_split': [4],
               'max_features':[2,4,6],
              'bootstrap': [True]
              }

rf_random_search = RandomizedSearchCV(RandomForestClassifier(class_weight ='balanced_subsample', random_state=0), param_distributions=param_dist,
                                   n_iter=10)
#Fit the Data
rf_random_search.fit(X_train2_matrix, y_train2)
print(rf_random_search.score(X_test_matrix, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.6190639486886391
-- Execution time: 629.9800851345062 seconds ---


In [22]:
rf_random_search.best_params_

{'n_estimators': 800,
 'min_samples_split': 4,
 'max_features': 4,
 'max_depth': 10,
 'criterion': 'entropy',
 'bootstrap': True}

In [23]:
print(classification_report(y_test, rf_random_search.predict(X_test_matrix)))

              precision    recall  f1-score   support

           1       0.43      0.63      0.51      7335
           2       0.24      0.14      0.17      4160
           3       0.25      0.34      0.29      5980
           4       0.29      0.27      0.28     11089
           5       0.81      0.77      0.79     50171

   micro avg       0.62      0.62      0.62     78735
   macro avg       0.41      0.43      0.41     78735
weighted avg       0.63      0.62      0.62     78735



## XGboost model

In [24]:
start_time = time.time()
param_dist = {'max_depth':[5,7,9],
              'subsample':[0.5,0.7,0.9],
              'colsample_bytree': [0.5,0.7,0.9],
            'colsample_bylevel':[0.5,0.7,0.9]
              }
#scale_pos_weight 
xgb_random_search = RandomizedSearchCV(xgb.XGBClassifier(learning_rate =.01, random_state=0), param_distributions=param_dist,
                                   n_iter=10)
#Fit the Data
xgb_random_search.fit(X_train2_matrix, y_train2)
print(xgb_random_search.score(X_test_matrix, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.6678732456975932
-- Execution time: 2954.1044821739197 seconds ---


In [25]:
xgb_random_search.best_params_

{'subsample': 0.7,
 'max_depth': 9,
 'colsample_bytree': 0.9,
 'colsample_bylevel': 0.7}

In [26]:
print(classification_report(y_test, xgb_random_search.predict(X_test_matrix)))

              precision    recall  f1-score   support

           1       0.69      0.19      0.29      7335
           2       0.30      0.01      0.01      4160
           3       0.47      0.09      0.15      5980
           4       0.53      0.09      0.15     11089
           5       0.67      0.99      0.80     50171

   micro avg       0.67      0.67      0.67     78735
   macro avg       0.53      0.27      0.28     78735
weighted avg       0.62      0.67      0.57     78735



## Modified models:

In [27]:
df2 = df.copy()
df2.head()

,Score,Text,Clean
0,5,I have bought several of the Vitality canned d...,i have bought several of the vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled as jumbo salted peanut...
2,4,This is a confection that has been around a fe...,this is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...,if you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...,great taffy at a great price. there was a wide...


In [28]:
#create training/test sets
X = df2.Clean
y = df2.Score

#split out training/test data 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

#split training set again to test best params with reduce execution times 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train,y_train, test_size=0.8, random_state=0)

In [29]:
vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             lowercase=True, #convert everything to lower case
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True, #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                             tokenizer=lemmatize_sen)


#Applying the vectorizer
tfidf_X_train2_matrix = vectorizer.fit_transform(X_train2)

tfidf_X_train2_matrix

<62988x25580 sparse matrix of type '<class 'numpy.float64'>'
	with 2853235 stored elements in Compressed Sparse Row format>

In [30]:
# Apply to X test
tfidf_X_test_matrix = vectorizer.transform(X_test)

tfidf_X_test_matrix

<78735x25580 sparse matrix of type '<class 'numpy.float64'>'
	with 3531318 stored elements in Compressed Sparse Row format>

## Logistic regression 2

In [31]:
start_time = time.time()
param_dist = {'penalty':['l1','l2'],
                'C':[1,100,1000]}

lr = LogisticRegression()

random_search = RandomizedSearchCV(LogisticRegression(class_weight = 'balanced',random_state=0, n_jobs= -1), param_distributions=param_dist, n_iter=5)
#Fit the Data
random_search.fit(tfidf_X_train2_matrix, y_train2)
print(random_search.score(tfidf_X_test_matrix, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to

0.6899091890518829
-- Execution time: 418.4213693141937 seconds ---


In [32]:
random_search.best_params_

{'penalty': 'l2', 'C': 1}

In [33]:
from sklearn.metrics import classification_report

print(classification_report(y_test, random_search.predict(tfidf_X_test_matrix)))

              precision    recall  f1-score   support

           1       0.57      0.65      0.61      7335
           2       0.24      0.25      0.25      4160
           3       0.32      0.34      0.33      5980
           4       0.39      0.30      0.34     11089
           5       0.84      0.86      0.85     50171

   micro avg       0.69      0.69      0.69     78735
   macro avg       0.47      0.48      0.47     78735
weighted avg       0.68      0.69      0.69     78735



## Random forest 2

In [36]:
start_time = time.time()
param_dist = {'n_estimators':[500,700,800],
                'criterion':['entropy'],
                'max_depth':[6,8,10],
                'min_samples_split': [4],
               'max_features':[1,2,4],
              'bootstrap': [True]
              }

rf_random_search = RandomizedSearchCV(RandomForestClassifier(class_weight ='balanced_subsample', random_state=0), param_distributions=param_dist,
                                   n_iter=10)
#Fit the Data
rf_random_search.fit(tfidf_X_train2_matrix, y_train2)
print(rf_random_search.score(tfidf_X_test_matrix, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.5877945005397853
-- Execution time: 663.2710225582123 seconds ---


In [37]:
rf_random_search.best_params_

{'n_estimators': 700,
 'min_samples_split': 4,
 'max_features': 4,
 'max_depth': 10,
 'criterion': 'entropy',
 'bootstrap': True}

In [38]:
print(classification_report(y_test, rf_random_search.predict(tfidf_X_test_matrix)))

              precision    recall  f1-score   support

           1       0.40      0.57      0.47      7335
           2       0.17      0.14      0.15      4160
           3       0.19      0.25      0.22      5980
           4       0.26      0.22      0.24     11089
           5       0.79      0.75      0.77     50171

   micro avg       0.59      0.59      0.59     78735
   macro avg       0.36      0.39      0.37     78735
weighted avg       0.60      0.59      0.59     78735



## XGB2

In [41]:
start_time = time.time()
param_dist = {'max_depth':[5,7,9],
              'subsample':[0.5,0.7,0.9],
              'colsample_bytree': [0.5,0.7,0.9],
            'colsample_bylevel':[0.5,0.7,0.9]
              }
#scale_pos_weight 
xgb_random_search = RandomizedSearchCV(xgb.XGBClassifier(learning_rate =.01, random_state=0), param_distributions=param_dist,
                                   n_iter=10)
#Fit the Data
xgb_random_search.fit(tfidf_X_train2_matrix, y_train2)
print(xgb_random_search.score(tfidf_X_test_matrix, y_test))

c:\users\shuaix\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.6674922207404586


In [42]:
xgb_random_search.best_params_

{'subsample': 0.7,
 'max_depth': 9,
 'colsample_bytree': 0.9,
 'colsample_bylevel': 0.9}

In [44]:
print(classification_report(y_test, xgb_random_search.predict(tfidf_X_test_matrix)))

              precision    recall  f1-score   support

           1       0.70      0.19      0.30      7335
           2       0.32      0.01      0.01      4160
           3       0.50      0.08      0.13      5980
           4       0.54      0.09      0.15     11089
           5       0.67      0.99      0.80     50171

   micro avg       0.67      0.67      0.67     78735
   macro avg       0.55      0.27      0.28     78735
weighted avg       0.62      0.67      0.57     78735

